# Prototype Class

# Prototype

In [1]:
import pandas as pd

data_path = "/home/thanawat-nawan/NSCLC-OncogenicDriver_and_ResistanceAlterations-ClinicalOutcome/Data/NSCLC Data Collection revised 14.05.2568.xlsx"
total_patient = pd.read_excel(data_path, sheet_name=0)

In [2]:
# Removing missing data and separate blood and tissue specimens and
# Exact columns that used in analysis

blood_specimen_list = ["Blood, PAXgene", "Blood, Streck", "Blood, Roche", "Effusion, Pleural", "FNA"]
tissue_specimen_list = ["Tissue, FFPE", "Tissue, Fresh", "Cytological slide", "Slide, stained"]

analysis_columns = ["Code", "type", "Age", "SEX", "test", "Diag", "cost", "Specimen Type", "Source", "%tumor", "Method1", "Result1", "Method2", "Result2", "Method3", "Result3", "Method4", "Result4", "Method5", "Result5"]

# Grouping diagnosis into 4 groups: Non-small cell lung cancer (NSCLC), Lung cancer, Metastasis, Squamous cell carcinoma
non_small_cell_lung_cancer_list = [
    "Non-small cell lung cancer", "Non-small cell carcinoma", "non-small cell lung cancer", "non-small cell carcinoma", "Non small cell lung cancer", "Recurrent Non-small cell lung cancer", "Large cell lung carcinoma", "Large cell carcinoma"]
lung_cancer_list = ["Lung carcinoma", "Lung adenocarcinoma", "Lung cancer", "Lung pleomorphic carcinoma"]
metastasis_list = ["Metastatic adenocarcinoma", "Metastatic carcinoma", "Metastatic cancer", "Metastatic brain cancer"]
squamous_cell_carcinoma = ["Squamous cell carcinoma", "squamous cell carcinoma", "Squamous cell lung cancer", "Squamous cell Carcinoma"]

In [3]:
from research_package.data_extraction.format_dataframe import FormatDataFrame
data = FormatDataFrame(total_patient)
data.remove_data_missing_with_reset_index()
data.filter_specimen(blood_specimen_list, tissue_specimen_list)
data.remove_unnecessary_data(analysis_columns)
data.filter_diagnosis(non_small_cell_lung_cancer_list, lung_cancer_list, metastasis_list, squamous_cell_carcinoma)
data.create_patient_dictionary()
normalized_df = data.create_normalize_dataframe()
normalized_df

The code at row 356 was not found in the dataframe.
The code at row 622 was not found in the dataframe.
The code at row 1391 was not found in the dataframe.
The code at row 1579 was not found in the dataframe.


,id,count_times,time_label,age,sex,cost,specimen_type,source,tumor_percentage,diagnosis_group,...,EGFR T790M mutation,Lung cancer actionable mutation panel,Lung cancer fusion,BRAF mutation,T790M mutation,KRAS reflex NRAS/BRAF mutation,Comprehensive EGFR mutation panel,"KRAS,NRAS and BRAF mutation",EGFR mutation and Lung cancer actionable mutation panel,KRAS mutation
0,b'gAAAAABoiyFjch2yEYdFo5k4SOmCiEF5FDeHTqSY5SzV...,1,time_1,54.0,Female,8890.0,Blood,Plasma,NaN,Lung cancer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,b'gAAAAABoiyFju-oPFUMp8A8-VOy8hRqSfpGwgUlANRbt...,2,time_1,60.0,Female,3590.0,Blood,Plasma,NaN,Non small cell lung cancer,...,[No T790M],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,b'gAAAAABoiyFjZsyfALBNv17o9mVpbaaG9_g5QB1thSSN...,2,time_2,60.0,Female,3590.0,Blood,Plasma,NaN,Lung cancer,...,[No T790M],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b'gAAAAABoiyFjrQepfsZu4BaR9D4PDzfyxDNzi0zpF1Js...,1,time_1,79.0,Female,3590.0,Blood,Plasma,NaN,Lung cancer,...,[No T790M],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,b'gAAAAABoiyFjPiKgcYHNnqvSIu-1GoY5C9ICrR0Yyyj-...,1,time_1,71.0,Female,7410.0,FFPE,Lung,30.0,Lung cancer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,b'gAAAAABoiyFjwBR-Zn4o0JEddf3_9r1cI_s2fX3eQYBo...,1,time_1,75.0,Female,7410.0,FFPE,Lung,20.0,Lung cancer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3495,b'gAAAAABoiyFjq29ST0DgyyEO3ACm_CXsJGdKycxz_3Qv...,1,time_1,60.0,Male,7410.0,FFPE,Bronchus tissue,5.0,Lung cancer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3496,b'gAAAAABoiyFj8yr88Z1Espyq4R1puCAMv3gxjAjmU1Ct...,1,time_1,49.0,Male,7410.0,FFPE,Lymph node,70.0,Lung cancer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3497,b'gAAAAABoiyFjucEYq6I_JI0OiYnAGdIiYXjkQvVqL_z0...,1,time_1,69.0,Female,7410.0,FFPE,Pleural nodule,95.0,Non small cell lung cancer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from research_package.count import Count
unique_count = Count(normalized_df)
unique_count.exact_unique_item()["EGFR mutation"]

{"['Negative']": 1294,
 'nan': 1026,
 "['BRAF-Negative']": 168,
 "['Ex 19 deletion ']": 456,
 "['L858R']": 375,
 "['L861Q']": 20,
 "['negative']": 21,
 "['G719X AND T790M']": 1,
 "['L858R AND T790M']": 14,
 "['Ex 19 deletion AND T790M']": 11,
 "['S768I and L858R']": 5,
 "['G719X']": 15,
 "['No T790M']": 22,
 "['T790M']": 3,
 "['S768I and G719X']": 4,
 "['L858R ']": 1,
 "['Ex20 Insertion']": 32,
 "['BRAF - Negative']": 12,
 "['Lung DNA - L858R \\nLung RNA - Negative']": 1,
 "['L858R, T790M AND Ex20 Insertion']": 1,
 "['BRAF - Positive V600E']": 1,
 "['G791A/C/s AND S768I']": 1,
 "['L861Q and G719X']": 2,
 "['Ex 19 deletion AND Ex20 Insertion']": 3,
 "['L858R AND Ex20 Insertion']": 1,
 "['S768I']": 6,
 "['G719A/C/S']": 2,
 "['Ex 19 deletion  AND G719X']": 1}

In [5]:
unique_count.exact_unique_item()["EGFR T790M mutation"]

{'nan': 2948,
 "['No T790M']": 210,
 "['Ex 19 deletion AND T790M']": 98,
 "['Ex 19 deletion ']": 30,
 "['L858R AND T790M']": 49,
 "['T790M']": 38,
 "['L858R']": 34,
 "['Negative']": 78,
 "['negative']": 1,
 "['S768I, G719X and T790M']": 1,
 "['L858R and T790M']": 2,
 "['L858R, T790M AND Ex20 Insertion']": 1,
 "['T790M ; L861Q ; G719X']": 1,
 "['Ex19Del, T790M AND Ex20 Insertion']": 1,
 "['S768I']": 2,
 "['Ex 19 deletion AND Ex20 Insertion AND T790M']": 1,
 "['G719X AND T790M']": 1,
 "['L861Q']": 2,
 "['G719A/C/S']": 1}

In [6]:
unique_count.exact_unique_item()["Lung cancer actionable mutation panel"]

{'nan': 3203,
 "['Lung DNA - KRAS G12V  AND EGFR amplification\\nLung RNA - Negative']": 1,
 "['Lung DNA - EGFR Amplification AND MET Amplification\\nLung RNA - Negative']": 1,
 "['Lung DNA - Negative\\nLung RNA - Negative']": 4,
 "['Lung DNA - L858R\\nLung RNA - Negative']": 5,
 "['Negative']": 167,
 "['Ex 19 deletion ']": 14,
 "['Lung DNA - Ex19del and MET-Amplication \\nLung RNA negative']": 2,
 "['EGFR - Ex 19 deletion \\nBRAF - Positive V600E']": 1,
 "['Lung DNA - MET Amplification \\n Lung RNA - Negative']": 1,
 "['Lung DNA - Ex 19 deletion\\n Lung RNA - Negative']": 3,
 "['Lung DNA - L858R\\n Lung RNA - Negative']": 2,
 "['Ex 19 deletion \\nEGFR - C797S']": 1,
 "['Lung DNA-EGFR L858R  \\nLung RNA-Negative']": 1,
 "['Lung DNA -KRAS G12D \\nLung RNA - Negative']": 1,
 "['Lung DNA -EGFR Ex19Del \\nLung RNA - Negative']": 1,
 "['Lung DNA - L858R / L833V  \\nLung RNA - Negative']": 1,
 "['L858R']": 19,
 "['Lung DNA - Ex19Del \\nLung RNA - Negative']": 1,
 "['Lung DNA - KRAS G12V \\nL

In [7]:
unique_count.exact_unique_item()["Lung cancer fusion"]

{'nan': 3157,
 '[nan]': 42,
 "['Negative']": 170,
 "['NTRK3 expression imbalance equivocal']": 4,
 "['Lung DNA - L861Q\\nLung RNA - Neg']": 1,
 "['Lung cancer fusion - Negative']": 1,
 "['ROS1 fusion']": 13,
 "['NTRK1 expression imbalance']": 2,
 "['Lung RNA - Negative']": 1,
 "['Fusion - Negative']": 2,
 "['Ex 19 deletion ']": 39,
 "['No T790M']": 1,
 "['L858R']": 41,
 "['MET exon 14 skipping']": 1,
 "['MET ex14 skipping']": 1,
 "['Ex 19 deletion AND T790M']": 1,
 "['Ex20 Insertion']": 4,
 "['NTRK3 expression imbalance: Equivocal']": 5,
 "['RET fuion']": 1,
 "['ALK expression imbalance']": 4,
 "['ALK fusion']": 3,
 "['ROS1 expression imbalance']": 1,
 "['RET fusion']": 1,
 "['L861Q']": 2,
 "['ROS1 fusion AND RET fusion']": 1}

In [8]:
unique_count.exact_unique_item()["BRAF mutation"]

{'nan': 3305,
 "['Negative']": 108,
 "['L858R']": 34,
 "['Ex 19 deletion ']": 30,
 "['BRAF-Negative']": 7,
 "['negative']": 5,
 "['L861Q and G719X']": 2,
 "['Ex20 Insertion']": 1,
 "['G719X']": 1,
 "['L858R AND Ex20 Insertion']": 1,
 "['L858R AND T790M']": 1,
 "['S768I']": 1,
 "['L861Q']": 1,
 "['Ex 19 deletion AND T790M']": 1,
 "['Ex 19 deletion AND Ex20 Insertion']": 1}

In [9]:
unique_count.exact_unique_item()["KRAS reflex NRAS/BRAF mutation"]

{'nan': 3497, "['KRAS-G12D']": 1, "['KRAS-G12C']": 1}

In [10]:
unique_count.exact_unique_item()["Comprehensive EGFR mutation panel"]

{'nan': 3453,
 "['Negative']": 24,
 "['Ex 19 deletion ']": 7,
 "['Ex 19 deletion AND MET']": 1,
 "['L858R and L833V']": 1,
 "['KRAS-G12C']": 2,
 "['EGFR-G598V']": 1,
 "['EGFR-Amplification ERBB2-Amplification']": 1,
 "['Ex 19 deletion AND T790M']": 1,
 "['Ex20 Insertion']": 1,
 "['EGFR - R776H']": 1,
 "['KRAS-G12A']": 1,
 "['L858R']": 1,
 "['KRAS-G12V']": 2,
 "['EGFR-Negative']": 1,
 "['ALK fusion']": 1}

In [11]:
unique_count.exact_unique_item()["KRAS,NRAS and BRAF mutation"]

{'nan': 3498, "['KRAS - G12V \\nNRAS/BRAF - Negative']": 1}

In [12]:
unique_count.exact_unique_item()["EGFR mutation and Lung cancer actionable mutation panel"]

{'nan': 3497, "['Negative']": 2}

In [13]:
unique_count.exact_unique_item()["KRAS mutation"]

{'nan': 3495, "['Negative']": 3, "['KRAS-G12D']": 1}